## CLASS DATA

In [26]:
import random 

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score ==3:
            return Sentiment.NEUTRAL
        else: 
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews 
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return[x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk 
        random.shuffle(self.reviews)

## LOAD DATA

In [27]:
import json

file_name = '/Users/rayya/PycharmProjects/Scikit-Learn/Books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        # need the raw data to convert into json
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))


# reviews[5].sentiment
# reviews[5].score
reviews[5].text


'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## PREP DATA (to have some of the dataset for training and some for the testing)

In [28]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size= 0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [29]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


##### Bag of words vectorization (CountVectorization)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0])

&#34;Cecilia&#34; is similar to a Jane Austen novel but not EXACTLY like a Jane Austen novel.  This is important to keep in mind when reading this book.  That being said, I was very pleased with this novel.  I often found myself wishing that there were more books like Jane Austen, and that is what I found in &#34;Cecilia.&#34;  The following are reasons why I liked it.1.  It's long.  Yes, I always wish for more, and this book is huge.2.  It has a lot of twists and turns.3.  Murphy's Law rules in this Frances Burney book; anything that can go wrong will go wrong in a hilarious sort of manner.4.  It's clean.  I don't like to read bad words and sex scenes.Frances Burney does a great job.  I think she puts in even more twists and turns than Jane Austen does.  I will be reading more of her work.  I just wish A&E; would make a mini-series of &#34;Cecilia,&#34; like they did with &#34;Pride and Prejudice.&#34;
  (0, 6078)	0.09031743381482515
  (0, 6116)	0.09580503970467166
  (0, 2229)	0.04108

## CLASSIFICATION

#### Linear SVM

In [31]:
from sklearn import svm 

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### DECISION TREE

In [32]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### NAIVE BAYES

In [33]:
# Import necessary libraries
from sklearn.naive_bayes import GaussianNB

# Assuming train_x_vectors and test_x_vectors are sparse matrices
# Convert sparse matrices to dense arrays
train_x_vectors_dense = train_x_vectors.toarray()
test_x_vectors_dense = test_x_vectors.toarray()

# Create a Gaussian Naive Bayes classifier
clf_gnb = GaussianNB()

# Train the model
clf_gnb.fit(train_x_vectors_dense, train_y)

# Make a prediction on the first test sample
prediction = clf_gnb.predict(test_x_vectors_dense[0].reshape(1, -1))

print(f'Predicted class for the first test sample: {prediction[0]}')


Predicted class for the first test sample: POSITIVE


#### LOGISTIC REGRESSION

In [34]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

## EVALUATION

In [35]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors_dense, test_y))
print(clf_log.score(test_x_vectors, test_y))


0.8076923076923077
0.6346153846153846
0.6610576923076923
0.8052884615384616


In [36]:
# F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels = [Sentiment.POSITIVE, Sentiment.NEGATIVE]
)
# f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]
# )
# f1_score(test_y, clf_gnb.predict(test_x_vectors_dense), average=None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]
# )
# f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]
# )

array([0.80582524, 0.80952381])

#### Qualitative testing 

In [37]:
test_set = ['it was amazing', "bad book do not buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

 ### TUNING OUR MODEL (WITH GRID SEARCH)

In [39]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel' : ('linear', 'rbf'),'C' : (1,4,6,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv = 5)
clf.fit(train_x_vectors, train_y)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 6, 8, 16, 32),
                         'kernel': ('linear', 'rbf')})

In [40]:
print(clf.score(test_x_vectors, test_y))


0.8100961538461539


## Saving Model

#### Save Model

In [55]:
import pickle

with open('./Sentiment Classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

#### Load Model 

In [58]:
with open('./Sentiment Classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [60]:
print(test_x[1])

loaded_clf.predict(test_x_vectors[1])

Beautiful book.  Can't put it down.


array(['POSITIVE'], dtype='<U8')